In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px

from keras.models import load_model

In [66]:
data = pd.read_csv('../data/datadaily/data_daily0.csv', index_col='Date_time_nr')

In [6]:
model=load_model('models/model1.keras')

In [26]:

def df_to_X_y(data, window_size=2):
    df_as_np = data.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np)-window_size):
        row = [r for r in df_as_np[i:i+window_size]]
        X.append(row)
        label = df_as_np[i+window_size][-1]
        y.append(label)
    return np.array(X), np.array(y)
    
def add_yearly_fourier(df): 
    ts = pd.to_datetime(df.index) 
    day_of_year = ts.dayofyear 
    T = 365.25 
    df = df.copy() 
    df["year_sin"] = np.sin(2 * np.pi * day_of_year / T) 
    df["year_cos"] = np.cos(2 * np.pi * day_of_year / T) 
    return df 

def encode_wind_direction_circular(df): 
    df = df.copy() 
    theta = np.deg2rad(df["wind_deg"] % 360) 
    df["wind_sin"] = np.sin(theta) 
    df["wind_cos"] = np.cos(theta)
    df.drop(columns=["wind_deg"], inplace=True) 
    return df 

def feature_pipeline(df):
    if 'P_avg' in df.columns:
        df_X = df.drop(columns=['P_avg'])
        df_y = df['P_avg']
        df_X = encode_wind_direction_circular(df_X) 
        df_X = add_yearly_fourier(df_X) 
        df = pd.concat([df_X, df_y], axis=1)
        return df
    
    else:
        df = encode_wind_direction_circular(df) 
        df = add_yearly_fourier(df) 
        return df

def predictions(data, model):
    #model = load_model('../models/model1.keras')
    data_ml = feature_pipeline(data)
    X  = df_to_X_y(data_ml)[0]
    array = model.predict(X).ravel()
    data_pred = pd.DataFrame({'P_avg_predictions': array}, index = pd.to_datetime(data_ml.index[:-2], unit='s' ) )
    return data_pred

In [27]:
predictions(data, model)

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


,P_avg_predictions
Date_time_nr,
2013-01-01,244.725235
2013-01-02,391.209991
2013-01-03,226.592651
2013-01-04,198.841370
2013-01-05,176.288269
...,...
2017-12-25,857.891357
2017-12-26,759.617371
2017-12-27,369.857910


# plotly

In [67]:
data1 = data[1461: ]
data1P = predictions(data[1461:],model)
data1.shape, data1P.shape


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


((365, 8), (363, 1))

In [ ]:
from datetime import datetime
def power_plot(data):
    fig = px.area(data1P, y="P_avg_predictions")
    return fig
    
power_plot(data1).show()

In [30]:
fig = px.line(data1P, y="P_avg_predictions")
fig.show()

In [31]:
data1.head()

,temp,pressure,humidity,wind_speed,wind_deg,rain_1h,snow_1h,P_avg
Date_time_nr,,,,,,,,
1483228800,-2.175833,1022.666667,99.416667,2.250000,206.250000,0.000000,0.000000,82.372778
1483315200,-0.355000,1023.583333,100.000000,2.443333,238.333333,0.000000,0.028333,-0.761458
1483401600,-1.379583,1027.500000,98.833333,2.591250,213.500000,0.000000,0.000000,-3.422986
1483488000,0.083333,1022.041667,100.000000,5.444583,277.916667,0.020833,0.009167,-4.777014
1483574400,-0.537083,1029.708333,87.916667,3.065417,168.750000,0.000000,0.000000,-5.425347


In [70]:
# wind power 
def dist_plot(data):
    fig = px.density_heatmap(data1, x="wind_speed", y="P_avg", marginal_x="histogram", marginal_y="histogram")
    return fig

hist_plot(data1).show()

In [35]:
fig = px.density_contour(data1, x="wind_speed", y="P_avg", marginal_x="histogram", marginal_y="histogram")
fig.show()

In [71]:
def hist_plot(data):   
    fig = px.histogram(data, x="P_avg")
    return fig
hist_plot(data).show()

In [41]:
mask = data['P_avg'] < 500
data[mask].shape

(1415, 8)

In [42]:
data.head()

,temp,pressure,humidity,wind_speed,wind_deg,rain_1h,snow_1h,P_avg
Date_time_nr,,,,,,,,
1356998400,5.314167,1012.458333,88.083333,5.037083,212.500000,0.332917,0.0,782.613541
1357084800,2.317500,1027.291667,93.625000,3.236250,222.500000,0.000000,0.0,160.549097
1357171200,4.582500,1037.041667,95.166667,5.037083,209.583333,0.041667,0.0,396.093055
1357257600,7.101250,1037.791667,97.666667,2.915417,246.250000,0.000000,0.0,117.150973
1357344000,5.635000,1034.791667,94.166667,2.014167,232.083333,0.000000,0.0,62.375833


In [78]:
def polar_wind(data): 
    fig = px.scatter_polar(data, r="P_avg", theta="wind_deg")
    return fig
polar_wind(data).show()

In [77]:
def plot1(data1):
    data1['datetime'] = pd.to_datetime(data1.index, unit='s').day_of_year
    fig = px.line_polar(data1, r="P_avg", theta='datetime')
    return fig
                    
plot1(data1).show()

C:\Users\stgadmin\AppData\Local\Temp\ipykernel_18048\763096633.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

